<a href="https://colab.research.google.com/github/diegohugo570/backup-python/blob/main/08_Curso_Python_M%C3%B3dulo_8_Forma%C3%A7%C3%A3o_Engenheiro_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Módulo 8: LangChain & Agentes (Engenharia Cognitiva)**

Link do curso: [clique aqui](https://dascia.academy/)

> **Objetivo:** Transformar o Python de uma linguagem de script em uma linguagem de **orquestração de raciocínio**. Você aprenderá a criar sistemas que não apenas "falam", mas que **planejam**, **agem** no mundo real (APIs, Banco de Dados) e **lembram** do passado.

---

## **Aula 1: O Ecossistema LangChain (Arquitetura)**

<div align="center">
  <img src="https://raw.githubusercontent.com/langchain-ai/langchain/e1d113ea84a2edcf4a7709fc5be0e972ea74a5d9/docs/static/svg/langchain_stack_062024.svg">
</div>

Antes de codar, precisamos entender onde estamos pisando. O ecossistema da LangChain cresceu e se dividiu em três pilares fundamentais.

1. **LangChain (O Orquestrador):** A biblioteca Python que conecta os modelos (OpenAI, Anthropic) ao seu código. É aqui que construímos as "Chains" (cadeias de pensamento).
2. **LangGraph (O Controle):** Para criar agentes complexos que precisam de loops, condicionais (if/else) e persistência de estado. É a evolução das "Chains" lineares.
3. **LangSmith (A Observabilidade):** Uma plataforma para debugar o que sua IA está pensando. Como o LLM é uma "caixa preta", o LangSmith permite ver os logs de cada etapa do raciocínio.

Neste curso, focaremos no **LangChain** para construção de ferramentas e agentes rápidos.

### **A Arquitetura Modular do LangChain**

<div align="center">
  <img src="https://python.langchain.ac.cn/assets/images/ecosystem_packages-32943b32657e7a187770c9b585f22a64.png">
</div>

Antes de instalar, é crucial entender que o LangChain moderno não é mais um "monolito". Ele foi refatorado em uma arquitetura de camadas para ser mais seguro e leve em produção.

Ao invés de instalar uma coisa gigante, montamos nosso ambiente como peças de Lego:

* **`langchain-core`**: A fundação. Define as interfaces padrão (o que é uma "Mensagem", o que é um "Prompt"). É a única dependência obrigatória para tudo funcionar.
* **Integrações (`langchain-openai`, `langchain-anthropic`)**: Os provedores de IA agora vivem em pacotes isolados. Isso significa que podemos atualizar a versão da OpenAI sem quebrar o resto do sistema.
* **`langchain`**: O orquestrador. Contém a lógica das "Chains", estratégias de busca (Retrieval) e a "cola" que une os componentes.
* **`langchain-community`**: Onde vivem as integrações mantidas pela comunidade (ferramentas de terceiros, vetores de bancos específicos, etc.).

É por isso que, no comando abaixo, não instalamos apenas `langchain`, mas também o conector específico `langchain-openai`.

In [ ]:
# Instalando o langchain-openai


In [ ]:
# Configurando as variáveis de ambiente (userdata e os)

# Configuração de Chaves (Use os Segredos do Colab ou input manual com getpass)


## **Aula 2: Chat Models**

Um "Chat Model" moderno não funciona apenas recebendo texto e cuspindo texto. Na verdade, não APENAS assim...

Ele pode operar também sobre uma **Lista de Mensagens** que representa o estado atual de uma conversa.

### **2.1 String vs. Lista de Mensagens (O Polimorfismo)**

O método `.invoke()` do LangChain é polimórfico.

1. **Entrada Simples (String):** Se você fizer `modelo.invoke("Oi")`, o LangChain converte isso automaticamente, "por baixo do capô", para `[HumanMessage(content="Oi")]`. É ótimo para testes rápidos.
2. **Entrada de um Chat (Lista de Mensagens):** Para enviar mensagens simulando uma conversação entre um humano e a IA, a utilização de listas de mensagens é a mais adequada.

In [ ]:
# Importando o ChatOpenAI e enviando uma mensagem


# Instanciando o modelo (gpt-4o-mini e temperatura zero)


# Enviando a primeira mensagem (string)


### **2.2 A Anatomia das Mensagens**

O LangChain padroniza a comunicação com qualquer modelo (seja OpenAI, Llama ou Anthropic) usando três classes principais:

#### **`SystemMessage` (Instrução Principal):**
- É a instrução que orienta o comportamento do modelo.
- *Ex:* "Você é um especialista em SQL que responde apenas com código, sem explicações."
- Diferente do `HumanMessage`, o modelo trata isso como uma configuração persistente, não como parte do diálogo.

In [ ]:
# Importando a classe SystemMessae


# Variáveis do aplicativo
# - Nome do usuário
# - Principal Dificuldade
# - Idade
# - Experiência em IA (Pouca, Média, Muita)


# Definindo a instrução principal


# Criando o System Message


#### **`HumanMessage` (O Input):**
* Representa a fala do usuário.
* Em modelos multimodais (como GPT-4o), o `content` dessa mensagem pode conter não só texto, mas também imagens em Base64 ou URLs.

In [ ]:
# Importando a classe HumanMessage


# Definindo a mensagem incial


# Criando o Human Message


#### **`AIMessage` (O Output):**
* É o que o modelo retorna.
* Não contém apenas o texto (`.content`), mas também metadados cruciais (`.response_metadata`) como: contagem de tokens gastos, motivo da parada (finish_reason) e **Logprobs** (que veremos mais a frente)

In [ ]:
# Enviando o System e Human Messages para obter o AI Message


# Inspecionando o AI Message


In [ ]:
# Fazendo append na lista de mensagens e adicionando um novo Human Messae


# Enviando nova lista de mensagens


# Inspecionando a resposta


### 🧠 **Exercícios de Fixação (Aula 2)**

**Exercício 2.1: Few-Shot Learning (Ensinando por Exemplos)**

Muitas vezes, apenas dar instruções no System Prompt não é suficiente; precisamos dar exemplos práticos para o modelo entender o padrão desejado. Isso se chama "Few-Shot Prompting".

1. Crie uma lista de mensagens que ensine o modelo a converter **gírias** em **linguagem corporativa ultra-formal**.
2. **SystemMessage:** "Você é um assistente corporativo que converte linguagem informal em formal."
3. **Exemplo 1 (Human):** "E aí mano, beleza?" -> **(AI):** "Olá, prezado colega. Como tem passado?"
4. **Exemplo 2 (Human):** "Tô fora dessa reunião." -> **(AI):** "Infelizmente, declino o convite para o alinhamento."
5. **Teste (Human):** "Bora fechar isso logo, tá demorando muito."
6. Invoque o modelo passando essa lista histórica e veja se ele segue o padrão no final.

In [ ]:
# Escreva seu código aqui


**Exercício 2.2: Contexto Manual (A Memória da IA)**

Modelos de chat são "stateless" (sem memória). Se você perguntar "Quem é o Batman?" e depois perguntar "E quem é o parceiro dele?", ele não saberá de quem você está falando na segunda vez, a menos que você envie o histórico completo.

1. Defina um `SystemMessage`.
2. Crie uma `HumanMessage` perguntando: "Quem escreveu Dom Casmurro?".
3. Invoque o modelo e guarde a resposta (o objeto `AIMessage`) numa variável `resposta_1`.
4. Agora, crie uma **nova lista** contendo:
* O System Message original.
* A pergunta original ("Quem escreveu...").
* A `resposta_1` que você guardou.
* Uma nova `HumanMessage`: "Em que ano foi publicado?".


5. Invoque o modelo com E sem essa lista completa. Se ele responder o ano correto (1899), significa que ele manteve o contexto.


In [ ]:
# Escreva seu código aqui


## **Aula 3: Logprobs e Perplexidade**

Um *Chat Model* moderno não gera texto “no escuro”. A cada token produzido, o modelo calcula uma **distribuição de probabilidades** sobre o vocabulário, da qual emergem métricas fundamentais como a **perplexidade**.

O foco dessa aula não é "prompt bonito", mas **leitura matemática do comportamento do modelo**.

### **3.1 Perplexidade e Logprobs (definição técnica)**

Considere uma sequência de tokens $(t_1, \dots, t_N)$ e um modelo que atribui probabilidades condicionais
$$
p_i = P(t_i \mid t_{<i})
$$

A **perplexidade** (PPL) é definida como a exponencial da *cross-entropy* média por token:

$$
\mathrm{PPL} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N}\log p_i\right)
$$

Se você tem **logprobs** diretamente (isto é, $(\ell_i = \log p_i)$), então:

$$
\mathrm{PPL} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N}\ell_i\right)
$$

Interpretação: PPL é o "número efetivo de escolhas equiprováveis" por token.  

Ex.: PPL = 10 sugere que, em média, o modelo estava tão incerto quanto escolher entre 10 tokens equiprováveis a cada passo.

In [ ]:
# Importando numpy


# Exemplo: logprobs de tokens (log natural)


# Cálculo da perplexidade


### **3.2 Duas implementações equivalentes (probs vs logprobs)**

A definição original da perplexidade usa **probabilidades** porque ela é, essencialmente, uma **média geométrica do inverso das probabilidades** atribuídas pelo modelo aos tokens observados.  

O uso do **inverso** é fundamental porque a perplexidade não mede confiança, mas **incerteza**: probabilidades altas indicam tokens fáceis de prever e devem contribuir pouco para a métrica, enquanto probabilidades baixas indicam surpresa e devem aumentar significativamente o valor final. Ao inverter \(p_i\), tokens improváveis passam a gerar valores grandes, penalizando o modelo de forma proporcional à dificuldade da predição.

A média geométrica é a escolha natural quando lidamos com **quantidades multiplicativas** (probabilidades condicionais ao longo de uma sequência), pois preserva a escala relativa e penaliza fortemente eventos improváveis.

Formalmente, para uma sequência de probabilidades $(p_1, \dots, p_N)$, a perplexidade é definida como:

1) **Via probabilidades (média geométrica)**

$$
\mathrm{PPL}
=
\left(\prod_{i=1}^{N}\frac{1}{p_i}\right)^{\frac{1}{N}}
$$

Essa expressão pode ser lida como:
> “Em média, o modelo se comporta como se estivesse escolhendo entre  
$(\mathrm{PPL})$ alternativas equiprováveis a cada token.”

---

2) **Via logprobs (forma computacionalmente estável)**

Para obter uma forma numericamente estável, aplicamos o logaritmo e exploramos suas propriedades:

$$
\log(\mathrm{PPL})
=
\log\!\left(\left(\prod_{i=1}^{N}\frac{1}{p_i}\right)^{\frac{1}{N}}\right)
$$

$$
=
\frac{1}{N}\log\!\left(\prod_{i=1}^{N}\frac{1}{p_i}\right)
=
\frac{1}{N}\sum_{i=1}^{N}\log\!\left(\frac{1}{p_i}\right)
$$

$$
=
-\frac{1}{N}\sum_{i=1}^{N}\log(p_i)
$$

Aplicando a função exponencial em ambos os lados, obtemos a forma final:

$$
\mathrm{PPL}
=
\exp\!\left(-\frac{1}{N}\sum_{i=1}^{N}\log(p_i)\right)
$$

Essa reformulação mantém o mesmo significado estatístico da média geométrica,  
mas é **muito mais estável numericamente**, razão pela qual é a forma usada na prática em modelos de linguagem.

In [ ]:
# Vamos criar um caso "realista" com np.full() de sequência longa com probabilidades pequenas e seus logs


In [ ]:
# Forma instável (vai dar inf por overflow em 1/prob, ou underflow dependendo do caminho)


# Analisando resultado


In [ ]:
# Forma estável


# Analisando resultado


### **3.3 Prova: limite inferior da perplexidade é 1**

Como $(0 < p_i \le 1)$, então $(\log(p_i) \le 0)$ (a base é 2.71828...)

Logo:

$$
-\frac{1}{N}\sum_{i=1}^{N}\log(p_i) \ge 0
$$

Aplicando exponencial:

$$
\mathrm{PPL} = \exp\left(-\frac{1}{N}\sum_{i=1}^{N}\log(p_i)\right) \ge \exp(0)=1
$$

**Quando dá exatamente 1?**

Somente se $(\log(p_i)=0)$ para todo $(i)$, isto é, $(p_i=1)$ para todo token.
Ou seja: **o modelo tem certeza absoluta do próximo token em cada passo (caso degenerado)**.

Implicação prática:

- $(\mathrm{PPL} \approx 1)$ significa previsibilidade extrema (geralmente texto trivial/repetitivo ou "teacher forcing" perfeito).
- Valores maiores indicam mais incerteza/surpresa média por token.


In [ ]:
# Criando uma função que recebe um array de logprobs e retorna a perplexidade


# Caso mínimo: p_i = 1 => log p_i = 0 => PPL = 1


### **3.4. Detalhes técnicos que importam**

#### **3.4.1 Base do log**

- Se você usa $log$ natural: PPL = $\exp(\cdot)$
- Se usa $\log_2$: PPL = $2^{(\cdot)}$. O valor muda de escala, mas a ordenação/comparação permanece consistente se você for coerente.


#### **3.4.2 Perplexidade vs “confiança”**

Logprob alto (próximo de 0) por token tende a reduzir PPL, mas:
- PPL é uma **média geométrica** do inverso das probs.
- Um único token muito improvável pode puxar PPL pra cima com força (isso é característica, não bug).

#### **3.4.3 Comparação só faz sentido com mesma tokenização e mesma distribuição**

Trocar tokenizer muda N e muda os $p_i$. Comparar PPL entre modelos/tokenizers diferentes pode ser bem enganoso.

#### **3.4.4. PPL não "prova alucinação", mas é um ótimo sinal**

PPL alta sugere que a sequência está fora do "caminho comum" do modelo (pode ser erro, pode ser criatividade, pode ser domínio raro etc.).

In [ ]:
# Demonstração do "um token ruim estraga a festa"


### **3.4. Calculando a perplexidade real**
Agora que já vimos toda a teoria matemática por trás dos logprobs e da perplexidade, vamos calcular a perplexidade real de uma resposta da IA.

In [ ]:
# Instanciando o modelo (com logprobs = True)


# Enviando mensagem pedindo um poema


In [ ]:
# Pegando a lista de logprobs


# Calculando a perplexidade


### **🧠 Exercícios de Fixação (Aula 3)**

**Exercício 3.1: Fato vs. Criatividade (O Teste de Incerteza)**

A perplexidade mede o quão "surpreso" o modelo fica com a própria resposta. Respostas factuais tendem a ter perplexidade baixa (o modelo tem certeza). Respostas criativas ou alucinações tendem a ter perplexidade alta.

1. Crie dois prompts:
* `prompt_fato`: "Qual é a capital da França?"
* `prompt_criativo`: "Invente uma nova cor que não existe e descreva o cheiro dela."


2. Invoque o modelo duas vezes: a primeira vez com `temperature=0` e a segunda com `temperature=0.8` para ambos.
3. Calcule a perplexidade de cada resposta.
4. Imprima os quatro valores e compare. Qual é maior?

In [ ]:
# Escreva seu código aqui


**Exercício 3.5.2: O """Detector de Alucinação""" (Threshold)**

Em sistemas críticos (como um bot médico ou jurídico), não podemos aceitar respostas duvidosas.

1. Crie uma função `verificar_confianca(resposta, limite=1.2) -> str`.
2. Ela deve calcular a perplexidade.
3. Se a perplexidade for **menor** que o limite, retorne "✅ Confiança Alta (Seguro)".
4. Se for **maior**, retorne "⚠️ Confiança Baixa (Risco de Alucinação)".
5. Teste a função com uma pergunta sem sentido (ex: "Qual o CPF do Pato Donald?").

In [ ]:
# Escreva seu código aqui


## **Aula 4: Structured Output (A Ponte Texto-Software)**

Softwares não consomem frases, consomem **JSON**.
O método `.with_structured_output` usa o **Pydantic** para forçar o LLM a preencher um formulário rígido. Se o modelo errar o tipo (ex: colocar texto onde deveria ser data), o Pydantic valida e o LangChain pode até pedir para ele corrigir automaticamente.

In [ ]:
# Importando classes do Pydantic e Typing


# Definindo schema do Pydantic para extrair de um e-mail:
# - Nome do remetente
# - Data de envio
# - Categoria (comercial, suporte, pessoal, spam)
# - Resumo (de até 10 palavras)



In [ ]:
# Instanciando o modelo


# Associando a classe ao modelo


In [ ]:
# E-mail exemplo


In [ ]:
# Analisando o resultado


### **🧠 Exercícios de Fixação (Aula 4)**

**Exercício 1: Triagem Automática de Suporte (Ticket Routing)**

Em empresas grandes, ler cada ticket de suporte para saber para quem mandar é caro. Sua tarefa é criar um sistema que lê a reclamação e retorna um objeto pronto para ser salvo no banco de dados do Jira/Zendesk.

1. Crie uma classe Pydantic `TicketSuporte` com os campos:
* `cot`: Deve ser uma string representando a cadeia de pensamento da IA para realizar a tarefa
* `setor`: Deve ser um `Literal` com as opções: "Financeiro", "Suporte Técnico", "Vendas".
* `prioridade`: Um inteiro de 1 (Baixa) a 5 (Crítica).
* `tags`: Uma lista de strings com palavras-chave do problema (ex: ["login", "senha", "erro 404"]).
* `requer_humano`: Booleano. Se o cliente estiver muito irritado, True. Caso contrário, False.


2. Passe a reclamação: *"Cara, estou tentando pagar minha fatura há 3 dias e o site dá erro 500! Vou processar vocês se cortarem meu serviço!"*
3. Imprima o objeto resultante e verifique se ele detectou corretamente o setor "Financeiro" e a prioridade alta.

In [ ]:
# Escreva seu código aqui


**Exercício 2: O Extrator de Notas Fiscais (OCR Inteligente)**

Você está construindo um app de gestão financeira. O usuário tira foto da nota (que vira texto via OCR), mas o texto vem sujo. Você precisa estruturar isso.

1. Crie uma classe `ItemCompra` com `produto` (str) e `valor` (float).
2. Crie uma classe principal `NotaFiscal` com:
* `nome_loja`: str
* `data`: str (formato YYYY-MM-DD)
* `itens`: uma `list[ItemCompra]` (aqui está o desafio: extrair uma lista de objetos!)
* `valor_total`: float


3. Passe o texto bagunçado:
*"Compra no Mercado do João, dia 12 de março de 2024. Levei 2 leites por 10 reais cada (total 20) e um pão por 5.90. Deu 25.90 tudo."*
4. Veja se o modelo consegue quebrar os itens individualmente dentro da lista.

In [ ]:
# Escreva seu código aqui


---

## **Aula 5: Tool Calling (Dando Braços à IA)**

LLMs vivem numa caixa fechada. Eles não sabem a temperatura atual nem acessar a internet. **Tools** são a ponte.

O fluxo de Tool Calling é:

1. **LLM decide:** "Preciso chamar a função `temperatura` com o argumento `cidade='sp'`".
2. **Pausa:** O LLM para de gerar texto e te devolve um objeto `tool_call`.
3. **Python Executa:** Você roda a função real.
4. **Retorno:** Você devolve o resultado para o LLM.
5. **Resposta Final:** O LLM lê o resultado e formula a frase final para o usuário.

In [ ]:
# Importando o decorator tool


# Criando a Ferramenta (consultar_temperatura)


In [ ]:
# Instanciando o modelo


# Associando ao modelo a ferramenta


In [ ]:
# Perguntando a temperatura atual de uma cidade


# Analisando a resposta


In [ ]:
# Ele pediu pra chamar alguma ferramenta? Qual? Quais os argumentos?


In [ ]:
# Adicionando a intenção ao histórico


**`ToolMessage` (O Retorno da Função):**
* Representa o *resultado* prático vindo do "mundo real" (do seu código Python) após o modelo solicitar uma ação.
* *Ex:* "22°C" (o retorno de `get_temperatura`) ou "Erro: Usuário não encontrado".
* **Ponto Chave:** Ele deve conter obrigatoriamente o `tool_call_id`. É assim que o modelo sabe conectar essa resposta específica ("22°C") à pergunta específica que ele fez anteriormente.

In [ ]:
# Criando fluxo de execução das tools


In [ ]:
# Obtendo a resposta final


### **🧠 Exercícios de Fixação (Aula 5)**

**Exercício 4.1: O Agendador de Salas (Gestão de Recursos)**

Você está criando o bot interno de uma empresa. Os funcionários vivem perguntando no Slack se as salas de reunião estão livres. O sistema sabe quem está perguntando (pelo user do Slack), mas não sabe **qual sala** e **qual horário** eles querem. A IA precisa extrair isso do texto.

1. Crie uma ferramenta `@tool` chamada `verificar_disponibilidade_sala`.
2. Ela deve receber dois argumentos obrigatórios:
* `nome_sala` (str): Pode ser "Azul", "Vermelha" ou "Executiva".
* `horario` (str): O horário desejado (ex: "14:00", "15:00").


3. Dentro da função, use este dicionário para simular a agenda do dia:
```python
AGENDA_HOJE = {
    "azul": ["09:00", "10:00", "14:00"], # Horários OCUPADOS
    "vermelha": ["11:00", "15:00"],
    "executiva": [] # Livre o dia todo
}

```


4. A lógica é:
* Se a sala não existir, informe ao agente que a sala não existe
* Se o horário estiver na lista, informe ao agente que aquele horário está ocupado
* Se não estiver, informe ao agente que aquele horário está disponível


5. Pergunte: *"A sala Azul está livre às 2 da tarde?"*.
6. Teste também: *"A executiva tá tranquila às 10:00?"*.
7. Veja se o modelo extrai corretamente os parâmetros ("azul"/"14:00" e "executiva"/"10:00") e responde de acordo.

In [ ]:
# Escreva seu código aqui


**Exercício 5.2: O Corretor de Seguros (Cálculo Determinístico)**

LLMs são péssimos em matemática financeira. Uma seguradora quer um bot que dê cotações rápidas de seguro auto, mas o cálculo precisa ser exato (Python), e não uma estimativa do GPT.

1. Crie uma ferramenta `@tool` chamada `calcular_seguro_auto`.
2. Ela deve receber: `idade` (int), `valor_carro` (float) e `tem_sinistro` (bool).
3. A lógica de negócio (Python) é:
* Preço base é 3% do valor do carro.
* Se `idade < 25`, adiciona 20% de risco sobre o preço base.
* Se `tem_sinistro` for True, adiciona 50% de risco sobre o preço base.


4. Faça o bind e pergunte: *"Tenho 22 anos, um carro de 100 mil reais e nunca bati. Quanto fica o seguro?"*.
5. Verifique se o modelo extraiu os argumentos certos (idade=22, valor=100000, sinistro=False) e se ele usou o valor exato retornado pela função na resposta final.

In [ ]:
# Escreva seu código aqui


---

## **Aula 6: Agentes (Automatizando o Raciocínio)**

Fazer o loop acima (if tool_calls...) manualmente é cansativo.
O **Agente** (`create_agent`) é uma abstração que faz esse loop (ReAct - Reason, Act) automaticamente até chegar na resposta ou atingir um limite de passos.

In [ ]:
# Instanciando a função create_agent, do módulo agents do Langchain


# Instanciando o modelo


# Usando as mesmas ferramentas anteriores e criando o agente


In [ ]:
# Executando um prompt


### **🧠 Exercícios de Fixação (Aula 6)**

**Exercício 6.1: O Assistente de Viagens (Agregação de Informação)**

Muitas vezes, uma resposta simples para o usuário exige consultar várias fontes diferentes. O usuário pergunta "Quanto custa ir para Paris?", e o Agente precisa consultar o aéreo E a hospedagem separadamente para somar.

1. Crie duas ferramentas:
* `cotar_passagem(destino: str)`: Retorna o preço do voo. Use um mock: `{"paris": 4500, "ny": 3000}`.
* `cotar_hotel(destino: str)`: Retorna o preço da diária. Use um mock: `{"paris": 1200, "ny": 1500}`.


2. Crie um Agente (`create_tool_calling_agent`) com essas duas tools.
3. Pergunte: *"Quanto fica uma viagem para Paris considerando a passagem e 3 diárias de hotel?"*.
4. **O Desafio:** O Agente deve ser capaz de:
1. Chamar `cotar_passagem("paris")`.
2. Chamar `cotar_hotel("paris")`.
3. Fazer a conta (4500 + 3*1200) sozinho ou usando uma calculadora interna.
4. Responder o valor final.

In [ ]:
# Escreva seu código aqui


**Exercício 6.2: O Agente de Reembolso (Lógica Condicional)**

Este é um clássico de Customer Experience (CX). O Agente precisa verificar uma regra de negócio (Data da Compra) antes de executar uma ação (Gerar Reembolso). Ele não pode gerar o reembolso se o prazo expirou.

1. Crie três ferramentas:
* `obter_data_compra(id_pedido: str)`: Retorna uma data simulada.
* Se ID for "PED-ANTIGO", retorne "2020-01-01".
* Se ID for "PED-NOVO", retorne a data de hoje (use `datetime.now()`).

* `calcular_dias(data: str)`: Retorna a diferença em número de dias da data fornecida em relação a hoje.


* `gerar_reembolso(id_pedido: str)`: Retorna "Reembolso processado com sucesso!".


2. Crie um Agente e dê a ele o seguinte System Prompt:
*"Você é um agente financeiro. Verifique a data da compra. Se a compra foi feita há mais de 7 dias, RECUSE o reembolso educadamente. Se for recente, processe o reembolso."*
3. Teste 1: *"Quero reembolso do pedido PED-ANTIGO"*. (O agente deve checar a data, calcular os dias e negar, **sem** chamar a ferramenta de gerar reembolso).
4. Teste 2: *"Quero reembolso do pedido PED-NOVO"*. (O agente deve checar a data, calcular os dias e, vendo que é menor do que 7, chamar a ferramenta de reembolso).


In [ ]:
# Escreva seu código aqui


## **Aula 7: Streaming (A Arte da Resposta em Tempo Real)**

Até agora, usamos `.invoke()`, que espera a resposta completa ficar pronta para entregar tudo de uma vez. Isso é ruim para o usuário, que fica encarando uma tela parada.

O **Streaming** entrega a resposta "token a token" (letra por letra), assim que é gerada. Isso cria a sensação de velocidade imediata.

### **7.1 Streaming Simples (Chat Models)**

Para modelos de chat, basta trocar `.invoke()` por `.stream()`. O retorno não é mais uma mensagem, é um **Iterador** (Generator) que cospe pedacinhos de texto (`chunks`).

In [ ]:
# Importando a biblioteca time e o ChatOpenAI


# Instanciando o modelo


# Iniciando o processo de straming


# Em vez de esperar tudo, recebemos pedaços
# Chunk é um pedaço da AIMessage (para cada chunk no stream, printamos o content (com end='' e flush = True))


### 🧠 **Exercícios de Fixação (Aula 7)**

**Exercício 7.1: O Efeito Máquina de Escrever**

Crie uma função `stream_texto(texto: str)` que recebe um texto longo pronto.
Dentro dela, simule um streaming "falso": percorra o texto letra por letra, imprima com `end=""`, dê um `time.sleep(0.05)` e faça `flush`. Isso ajuda a entender a UX do streaming.

In [ ]:
# Escreva seu código aqui


**Exercício 7.2: Streaming com Stop**

Use o `llm.stream("Conte até 100")`.
Crie um loop que imprime os tokens. Adicione uma lógica: se o número "10" aparecer no chunk, pare o streaming (`break`) imediatamente e imprima "🚫 Interrompido pelo filtro de segurança.".

In [ ]:
# Escreva seu código aqui


---

## **🚀 Próximo Passo: Projeto no Cursor**

Agora você tem o poder de criar "seres digitais".

**Vamos para a IDE Cursor construir o "The CLI Assistant".**

Neste projeto final, vamos elevar a barra. Além da memória e das ferramentas básicas, você implementará uma ferramenta de **Integração de API Real**.

### **O Desafio Extra: PokéAPI**

Você deve criar uma tool que usa a biblioteca `requests` para consultar a API pública do Pokémon (`https://pokeapi.co/api/v2/pokemon/{nome}`).
Seu assistente deverá ser capaz de responder:
*"Que horas são e qual é a altura do Pikachu?"*
Ele terá que chamar `get_current_time`, depois `get_pokemon_info`, e consolidar tudo.

**Estrutura do Projeto:**

* `cli_assistant.py`: A classe que gerencia o histórico e o Agente.
* `tools.py`: Onde você escreverá a função `get_pokemon_info` usando `requests`.
* `main.py`: O loop de interação com o usuário.

Vá para o Cursor. É hora de virar um Engenheiro de IA.